## Bandit-IV

In [1]:
import numpy as np
import numpy.random as ra
import numpy.linalg as la
from tqdm import tqdm
import matplotlib.pyplot as plt
# import os
# print(os.getcwd())

In [4]:
def runit(Z_dist, 
          X_dist,
          Y_dist,
          beta_star,
          gamma_star, 
          gamma_z, 
          gamma_x, 
          ):
      
      np.random.seed(42)

    
      nb_exp, T, nb_arms, d = X_dist.shape
      _, _, _, k = Z_dist.shape

      #result tracking
      regret = np.zeros((nb_exp,T))
      pred_norms = np.zeros((nb_exp,T))
      x_inv_norms = np.zeros((nb_exp,T))
      norm_beta_hat__beta = np.zeros((nb_exp, T))
      beta_hat_rec = np.zeros((nb_exp,T,d))
      GammaHat_rec = np.zeros((nb_exp,T, k, d))

      for it in range(nb_exp): 

            #intializing estimated parameter matrics
            U_0 = np.eye(k)*gamma_z         # k x k
            V_0 = np.zeros((k,d))           # k x d
            W_0 = np.eye(d)*gamma_x         # d x d
            Q_0 = np.zeros((d,1))           # d x 1

            #initial pull 
            idx = np.random.choice(np.arange(Z_dist.shape[0]))          #choose arm at random



      for t in range(1,T):

            U_t = U_0 + z @ z.T
            V_t = V_0 + z @ x.T     
            Gamma_est = np.linalg.inv(U_t) @ V_t          #shape : d x k
            X_est = (Z.T @ Gamma_est).T         #shape : d x (n rounds so far..) (different from paper)
            W_t = W_0 + X_est @ X_est.T         #operation is other way in paper but that yield 1x1 (not dxd)
            Q_t = Q_0 + (X_est @ Y)[...,None]       #had to remove transpose
            Beta_est = inv(W_t) @ Q_t           #shape : k x 1

    return regret, pred_norms, x_inv_norms, beta_hat_rec, ThetaHat_rec, norm_beta_hat__beta

In [2]:
S, sigma_1, sigma_2 = 1, 0.01, 0.0001
nb_exp, T, n_a, d = 100, int(1e3), 100, 20

theta_star = np.random.randn(d,d); #  theta_star *= 1/np.linalg.norm(theta_star) #maybe this can be changed
inv_theta_star = np.linalg.inv(theta_star)
beta_star = np.random.multivariate_normal([10 for i in range(d)], np.eye(d)).reshape(-1,) ; beta_star *= S/np.linalg.norm(beta_star)

lamda = 1 ; lamda_IV = 1e-3
delta = 0.001
seed = 1996
S_hat = S
pulled_indeces = np.zeros((nb_exp,T))

algorithms = ['banditiv']

# FIRST STAGE
# X_max = 100
Z = np.random.randn(nb_exp, T, n_a, d) #; Z /= np.max(np.linalg.norm(Z,axis=2))/X_max
eps = sigma_1*np.random.randn(nb_exp, T, n_a, d)
# eps = np.zeros((T,d))
X = np.dot(Z, theta_star) + eps

# SECOND STAGE
ee = d//4
eta_tilde = np.random.randn(nb_exp, T, n_a) 
# eta = (eta_tilde + np.sum(eps[:,:,:,:ee]))/(1+ee*sigma_1)*sigma_2
eta = sigma_2 * (eta_tilde + eps[:,:,:,:ee].sum(axis=-1))
X = X / np.max(np.linalg.norm(X, axis=3, keepdims = True), axis=2, keepdims=True) 
Y = np.dot(X, beta_star) + eta

In [5]:
regrets = np.zeros((nb_exp,len(algorithms),T))
pred_norms = np.zeros((nb_exp,len(algorithms),T))
x_inv_norms = np.zeros((nb_exp,len(algorithms),T))
# beta_hat_ridge = np.zeros((nb_exp,T,d))
beta_hat = np.zeros((nb_exp, len(algorithms), T,d))
theta_hat = np.zeros((nb_exp, len(algorithms), T, d, d))
beta_error = np.zeros((nb_exp, len(algorithms), T))

for i,a in enumerate(algorithms):
    regrets[:,i,:], pred_norms[:,i,:], x_inv_norms[:,i,:],\
         beta_hat[:, i, :,:] , theta_hat[:, i, :,:, :], beta_error[:,i,:]\
             = runit(Z, X, Y, beta_star, S_hat, sigma_1, sigma_2, lamda, lamda_IV, delta, seed, pulled_indeces, a)

In [6]:
regrets

array([[[0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.]]])